# Understanding Embedding Spaces

## Imports and definitions

In [49]:
import json
from pprint import pprint
! pip install av
! pip install yt-dlp
import yt_dlp
import av
import av.datasets

## Donwloading the videos

In [50]:
with open('activity_net.v1-3.min.json', 'r') as json_data:
    data = json.load(json_data)

In [51]:
# Extract video information
videos = data.get('database', {})

# Sort videos by the number of moments (annotations)
sorted_videos = sorted(videos.items(), key=lambda x: len(x[1].get('annotations', [])), reverse=True)

# Select the top 10 videos
top_15_videos = sorted_videos[:15]

# Print the top 10 video IDs and their moment counts
for video_id, video_info in top_15_videos:
    print(f"Video ID: {video_id}, Moments: {len(video_info.get('annotations', []))}")

Video ID: o1WPnnvs00I, Moments: 23
Video ID: oGwn4NUeoy8, Moments: 23
Video ID: VEDRmPt_-Ms, Moments: 20
Video ID: qF3EbR8y8go, Moments: 19
Video ID: DLJqhYP-C0k, Moments: 18
Video ID: t6f_O8a4sSg, Moments: 18
Video ID: 6gyD-Mte2ZM, Moments: 18
Video ID: jBvGvVw3R-Q, Moments: 18
Video ID: PJ72Yl0B1rY, Moments: 17
Video ID: QHn9KyE-zZo, Moments: 17
Video ID: 9-yueOtwiL8, Moments: 17
Video ID: od9EdcDcByA, Moments: 17
Video ID: U0qUFAPUg_Y, Moments: 16
Video ID: -ap649M020k, Moments: 15
Video ID: lxMSX0Y0b8I, Moments: 15


In [52]:
def download_video(url, output_path='downloads/'):
    ydl_opts = {
        'outtmpl': f'{output_path}/%(title)s.%(ext)s',
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

In [ ]:
count = 0
for video_id, video_info in top_15_videos:
    video_url = video_info.get('url')
    if video_url and "youtube.com" in video_url and count < 10:
        try:
            download_video(video_url)
            print(f"Downloaded: {video_id}")
            count += 1
        except Exception as e:
            print(f"Failed to download {video_id}: {e}")
    else:
        print(f"Invalid or unsupported URL for video ID or already gor 10 videos: {video_id}")

[youtube] Extracting URL: https://www.youtube.com/watch?v=o1WPnnvs00I
[youtube] o1WPnnvs00I: Downloading webpage
[youtube] o1WPnnvs00I: Downloading tv client config
[youtube] o1WPnnvs00I: Downloading player ac290d0b-main
[youtube] o1WPnnvs00I: Downloading tv player API JSON
[youtube] o1WPnnvs00I: Downloading ios player API JSON
[youtube] o1WPnnvs00I: Downloading m3u8 information


[info] o1WPnnvs00I: Downloading 1 format(s): 18
[download] Destination: downloads\Irish March (Fife & Flute) – Wouter Kellerman (Live).mp4
[download] 100% of   10.19MiB in 00:00:21 at 480.13KiB/s 
Downloaded: o1WPnnvs00I
[youtube] Extracting URL: https://www.youtube.com/watch?v=oGwn4NUeoy8
[youtube] oGwn4NUeoy8: Downloading webpage
[youtube] oGwn4NUeoy8: Downloading tv client config
[youtube] oGwn4NUeoy8: Downloading player ac290d0b-main
[youtube] oGwn4NUeoy8: Downloading tv player API JSON
[youtube] oGwn4NUeoy8: Downloading ios player API JSON
[youtube] oGwn4NUeoy8: Downloading m3u8 information


[info] oGwn4NUeoy8: Downloading 1 format(s): 18
[download] Destination: downloads\Jessica Ryckewaert plays Congas Drums Mezza Voce, Rhythm Concert.mp4
[download] 100% of    5.66MiB in 00:00:12 at 481.19KiB/s   
Downloaded: oGwn4NUeoy8
[youtube] Extracting URL: https://www.youtube.com/watch?v=VEDRmPt_-Ms
[youtube] VEDRmPt_-Ms: Downloading webpage
[youtube] VEDRmPt_-Ms: Downloading tv client config
[youtube] VEDRmPt_-Ms: Downloading player ac290d0b-main
[youtube] VEDRmPt_-Ms: Downloading tv player API JSON
[youtube] VEDRmPt_-Ms: Downloading ios player API JSON
[youtube] VEDRmPt_-Ms: Downloading m3u8 information


[info] VEDRmPt_-Ms: Downloading 1 format(s): 18
[download] Destination: downloads\Raquel.Pinto.Estrela.Moitense.Tumbling.2013.mp4
[download] 100% of   11.13MiB in 00:00:23 at 483.10KiB/s   
Downloaded: VEDRmPt_-Ms
[youtube] Extracting URL: https://www.youtube.com/watch?v=qF3EbR8y8go
[youtube] qF3EbR8y8go: Downloading webpage
[youtube] qF3EbR8y8go: Downloading tv client config
[youtube] qF3EbR8y8go: Downloading player ac290d0b-main
[youtube] qF3EbR8y8go: Downloading tv player API JSON
[youtube] qF3EbR8y8go: Downloading ios player API JSON
[youtube] qF3EbR8y8go: Downloading m3u8 information


[info] qF3EbR8y8go: Downloading 1 format(s): 18
[download] Destination: downloads\Chinese Brush Painting.mp4
[download] 100% of    9.50MiB in 00:00:20 at 482.23KiB/s 
Downloaded: qF3EbR8y8go
[youtube] Extracting URL: https://www.youtube.com/watch?v=DLJqhYP-C0k
[youtube] DLJqhYP-C0k: Downloading webpage
[youtube] DLJqhYP-C0k: Downloading tv client config
[youtube] DLJqhYP-C0k: Downloading player ac290d0b-main
[youtube] DLJqhYP-C0k: Downloading tv player API JSON
[youtube] DLJqhYP-C0k: Downloading ios player API JSON
[youtube] DLJqhYP-C0k: Downloading m3u8 information


...

[info] jBvGvVw3R-Q: Downloading 1 format(s): 18
[download] Destination: downloads\The King of American Weightlifting Visits Cal Strength.mp4
[download] 100% of   13.51MiB in 00:00:30 at 448.87KiB/s   
Downloaded: jBvGvVw3R-Q
[youtube] Extracting URL: https://www.youtube.com/watch?v=PJ72Yl0B1rY
[youtube] PJ72Yl0B1rY: Downloading webpage
[youtube] PJ72Yl0B1rY: Downloading tv client config
[youtube] PJ72Yl0B1rY: Downloading player ac290d0b-main
[youtube] PJ72Yl0B1rY: Downloading tv player API JSON
[youtube] PJ72Yl0B1rY: Downloading ios player API JSON


ERROR: [youtube] PJ72Yl0B1rY: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Failed to download PJ72Yl0B1rY: ERROR: [youtube] PJ72Yl0B1rY: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
[youtube] Extracting URL: https://www.youtube.com/watch?v=QHn9KyE-zZo
[youtube] QHn9KyE-zZo: Downloading webpage
[youtube] QHn9KyE-zZo: Downloading tv client config
[youtube] QHn9KyE-zZo: Downloading player ac290d0b-main
[youtube] QHn9KyE-zZo: Downloading tv player API JSON
[youtube] QHn9KyE-zZo: Downloading ios player API JSON
[youtube] QHn9KyE-zZo: Downloading m3u8 information


[info] QHn9KyE-zZo: Downloading 1 format(s): 18
[download] Destination: downloads\London Slacklining.mp4
[download] 100% of   16.67MiB in 00:00:37 at 455.21KiB/s 
Downloaded: QHn9KyE-zZo
[youtube] Extracting URL: https://www.youtube.com/watch?v=9-yueOtwiL8
[youtube] 9-yueOtwiL8: Downloading webpage
[youtube] 9-yueOtwiL8: Downloading tv client config
[youtube] 9-yueOtwiL8: Downloading player ac290d0b-main
[youtube] 9-yueOtwiL8: Downloading tv player API JSON
[youtube] 9-yueOtwiL8: Downloading ios player API JSON
[youtube] 9-yueOtwiL8: Downloading m3u8 information


[info] 9-yueOtwiL8: Downloading 1 format(s): 18
[download] Destination: downloads\Shot-Put, Discus, Javelin.mp4
[download] 100% of   13.35MiB in 00:00:28 at 476.78KiB/s   
Downloaded: 9-yueOtwiL8
Invalid or unsupported URL for video ID: od9EdcDcByA
Invalid or unsupported URL for video ID: U0qUFAPUg_Y
Invalid or unsupported URL for video ID: -ap649M020k
Invalid or unsupported URL for video ID: lxMSX0Y0b8I


In [54]:
# content = av.datasets.curated("pexels/time-lapse-video-of-night-sky-857195.mp4")
# with av.open(content) as container:
#     # Signal that we only want to look at keyframes.
#     stream = container.streams.video[0]
#     stream.codec_context.skip_frame = "NONKEY"

#     for i, frame in enumerate(container.decode(stream)):
#         print(frame)
#         frame.to_image().save(f"night-sky.{i:04d}.jpg", quality=80)

## Storing video data